# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f9258b230d0>
├── 'a' --> tensor([[-1.5016,  0.2365,  0.1983],
│                   [-0.5728, -2.9007, -0.1820]])
└── 'x' --> <FastTreeValue 0x7f9258b531f0>
    └── 'c' --> tensor([[-8.3100e-01, -1.1192e+00, -2.7592e-01,  7.4312e-01],
                        [ 1.3845e-03,  1.1285e+00,  1.3368e+00, -1.2128e+00],
                        [-1.5134e+00,  5.6269e-01,  2.8417e-01,  1.8992e-01]])

In [4]:
t.a

tensor([[-1.5016,  0.2365,  0.1983],
        [-0.5728, -2.9007, -0.1820]])

In [5]:
%timeit t.a

60.3 ns ± 0.0472 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f9258b230d0>
├── 'a' --> tensor([[ 0.7080, -0.5880, -0.1838],
│                   [ 0.5535, -1.5117, -1.2745]])
└── 'x' --> <FastTreeValue 0x7f9258b531f0>
    └── 'c' --> tensor([[-8.3100e-01, -1.1192e+00, -2.7592e-01,  7.4312e-01],
                        [ 1.3845e-03,  1.1285e+00,  1.3368e+00, -1.2128e+00],
                        [-1.5134e+00,  5.6269e-01,  2.8417e-01,  1.8992e-01]])

In [7]:
%timeit t.a = new_value

57.1 ns ± 0.0315 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.5016,  0.2365,  0.1983],
               [-0.5728, -2.9007, -0.1820]]),
    x: Batch(
           c: tensor([[-8.3100e-01, -1.1192e+00, -2.7592e-01,  7.4312e-01],
                      [ 1.3845e-03,  1.1285e+00,  1.3368e+00, -1.2128e+00],
                      [-1.5134e+00,  5.6269e-01,  2.8417e-01,  1.8992e-01]]),
       ),
)

In [10]:
b.a

tensor([[-1.5016,  0.2365,  0.1983],
        [-0.5728, -2.9007, -0.1820]])

In [11]:
%timeit b.a

61 ns ± 0.0327 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7604,  0.2665, -0.2677],
               [-0.8147, -0.3625,  0.8706]]),
    x: Batch(
           c: tensor([[-8.3100e-01, -1.1192e+00, -2.7592e-01,  7.4312e-01],
                      [ 1.3845e-03,  1.1285e+00,  1.3368e+00, -1.2128e+00],
                      [-1.5134e+00,  5.6269e-01,  2.8417e-01,  1.8992e-01]]),
       ),
)

In [13]:
%timeit b.a = new_value

490 ns ± 0.122 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

833 ns ± 0.268 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 29.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 1.59 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 309 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f91a9acde50>
├── 'a' --> tensor([[[-1.5016,  0.2365,  0.1983],
│                    [-0.5728, -2.9007, -0.1820]],
│           
│                   [[-1.5016,  0.2365,  0.1983],
│                    [-0.5728, -2.9007, -0.1820]],
│           
│                   [[-1.5016,  0.2365,  0.1983],
│                    [-0.5728, -2.9007, -0.1820]],
│           
│                   [[-1.5016,  0.2365,  0.1983],
│                    [-0.5728, -2.9007, -0.1820]],
│           
│                   [[-1.5016,  0.2365,  0.1983],
│                    [-0.5728, -2.9007, -0.1820]],
│           
│                   [[-1.5016,  0.2365,  0.1983],
│                    [-0.5728, -2.9007, -0.1820]],
│           
│                   [[-1.5016,  0.2365,  0.1983],
│                    [-0.5728, -2.9007, -0.1820]],
│           
│                   [[-1.5016,  0.2365,  0.1983],
│                    [-0.5728, -2.9007, -0.1820]]])
└── 'x' --> <FastTreeValue 0x7f91a9acdee0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.8 µs ± 1.01 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f91a2ea53d0>
├── 'a' --> tensor([[-1.5016,  0.2365,  0.1983],
│                   [-0.5728, -2.9007, -0.1820],
│                   [-1.5016,  0.2365,  0.1983],
│                   [-0.5728, -2.9007, -0.1820],
│                   [-1.5016,  0.2365,  0.1983],
│                   [-0.5728, -2.9007, -0.1820],
│                   [-1.5016,  0.2365,  0.1983],
│                   [-0.5728, -2.9007, -0.1820],
│                   [-1.5016,  0.2365,  0.1983],
│                   [-0.5728, -2.9007, -0.1820],
│                   [-1.5016,  0.2365,  0.1983],
│                   [-0.5728, -2.9007, -0.1820],
│                   [-1.5016,  0.2365,  0.1983],
│                   [-0.5728, -2.9007, -0.1820],
│                   [-1.5016,  0.2365,  0.1983],
│                   [-0.5728, -2.9007, -0.1820]])
└── 'x' --> <FastTreeValue 0x7f9258b60f10>
    └── 'c' --> tensor([[-8.3100e-01, -1.1192e+00, -2.7592e-01,  7.4312e-01],
                        [ 1.3845e-03,  1.1285e+00,  1.3368e+00,

In [23]:
%timeit t_cat(trees)

29.8 µs ± 98 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.4 µs ± 73.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-8.3100e-01, -1.1192e+00, -2.7592e-01,  7.4312e-01],
                       [ 1.3845e-03,  1.1285e+00,  1.3368e+00, -1.2128e+00],
                       [-1.5134e+00,  5.6269e-01,  2.8417e-01,  1.8992e-01]],
              
                      [[-8.3100e-01, -1.1192e+00, -2.7592e-01,  7.4312e-01],
                       [ 1.3845e-03,  1.1285e+00,  1.3368e+00, -1.2128e+00],
                       [-1.5134e+00,  5.6269e-01,  2.8417e-01,  1.8992e-01]],
              
                      [[-8.3100e-01, -1.1192e+00, -2.7592e-01,  7.4312e-01],
                       [ 1.3845e-03,  1.1285e+00,  1.3368e+00, -1.2128e+00],
                       [-1.5134e+00,  5.6269e-01,  2.8417e-01,  1.8992e-01]],
              
                      [[-8.3100e-01, -1.1192e+00, -2.7592e-01,  7.4312e-01],
                       [ 1.3845e-03,  1.1285e+00,  1.3368e+00, -1.2128e+00],
                       [-1.5134e+00,  5.6269e-01,  2.8417e-01,  1.8992e-01]],
      

In [26]:
%timeit Batch.stack(batches)

78.1 µs ± 129 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-8.3100e-01, -1.1192e+00, -2.7592e-01,  7.4312e-01],
                      [ 1.3845e-03,  1.1285e+00,  1.3368e+00, -1.2128e+00],
                      [-1.5134e+00,  5.6269e-01,  2.8417e-01,  1.8992e-01],
                      [-8.3100e-01, -1.1192e+00, -2.7592e-01,  7.4312e-01],
                      [ 1.3845e-03,  1.1285e+00,  1.3368e+00, -1.2128e+00],
                      [-1.5134e+00,  5.6269e-01,  2.8417e-01,  1.8992e-01],
                      [-8.3100e-01, -1.1192e+00, -2.7592e-01,  7.4312e-01],
                      [ 1.3845e-03,  1.1285e+00,  1.3368e+00, -1.2128e+00],
                      [-1.5134e+00,  5.6269e-01,  2.8417e-01,  1.8992e-01],
                      [-8.3100e-01, -1.1192e+00, -2.7592e-01,  7.4312e-01],
                      [ 1.3845e-03,  1.1285e+00,  1.3368e+00, -1.2128e+00],
                      [-1.5134e+00,  5.6269e-01,  2.8417e-01,  1.8992e-01],
                      [-8.3100e-01, -1.1192e+00, -2.7592e-01,  7.43

In [28]:
%timeit Batch.cat(batches)

141 µs ± 261 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

306 µs ± 1.12 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
